In [3]:
import random
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from constants import PROVINCIAS
from data.io import save_df_to_parquet
from scrapper import IdealistaScraper

In [22]:
id_pisos_comunidad_autonoma = []

for provincia in PROVINCIAS:
    # Definir los headers
    headers_provincia = {
        "User-Agent": UserAgent().random,
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Upgrade-Insecure-Requests": "1",
        "Referer": "https://www.google.com/",
    }

    for pag in range(1, 7):
        # Enlace relacionado con todos los pisos en renta para la provincia de interés.
        # Solo resultados de la primera página.
        url_provincia = f"https://www.idealista.com/alquiler-viviendas/{provincia}-provincia/pagina-{str(pag)}.htm"

        # Enviar la petición y ver la respuesta
        response_provincia = requests.get(url_provincia, headers=headers_provincia)

        if response_provincia.status_code == 200:
            # Analizar el contenido HTML usando BeautifulSoup
            soup = BeautifulSoup(response_provincia.content, "html.parser")

            # Extraer los tags que indican los id de los pisos publicados
            articles = soup.find_all("article", attrs={"data-element-id": True})
            id_pisos_publicados_provincia = [
                article["data-element-id"] for article in articles
            ]

            # Extender la lista de ids
            id_pisos_comunidad_autonoma.extend(id_pisos_publicados_provincia)

            # Agregar un delay para imitar un comportamiento humano
            time.sleep(random.uniform(1, 3))
            print(f"Scrapping de ids para la provincia de: {provincia} página {pag}")
        else:
            raise Exception(
                f"Scrapping fallido con código de respuesta {provincia} página {pag}"
                f"para la provincia: {provincia}"
            )

Scrapping de ids para la provincia de: barcelona página 1
Scrapping de ids para la provincia de: barcelona página 2
Scrapping de ids para la provincia de: barcelona página 3
Scrapping de ids para la provincia de: barcelona página 4
Scrapping de ids para la provincia de: barcelona página 5
Scrapping de ids para la provincia de: barcelona página 6
Scrapping de ids para la provincia de: girona página 1
Scrapping de ids para la provincia de: girona página 2
Scrapping de ids para la provincia de: girona página 3
Scrapping de ids para la provincia de: girona página 4
Scrapping de ids para la provincia de: girona página 5
Scrapping de ids para la provincia de: girona página 6
Scrapping de ids para la provincia de: lleida página 1
Scrapping de ids para la provincia de: lleida página 2
Scrapping de ids para la provincia de: lleida página 3
Scrapping de ids para la provincia de: lleida página 4
Scrapping de ids para la provincia de: lleida página 5
Scrapping de ids para la provincia de: lleida p

In [25]:
print("Número de pisos para hacer scrapping:", len(id_pisos_comunidad_autonoma))

Número de pisos para hacer scrapping: 720


In [48]:
info_pisos = []
contador = 0

# Extracción de la info de todos los pisos en la primera página
for piso in id_pisos_comunidad_autonoma[252:]:

    # Definir la url según el id
    url_piso = f"https://www.idealista.com/inmueble/{piso}/"

    # Generar
    scraper = IdealistaScraper(url_piso)
    data_piso = scraper.scrape()

    # Agregar el id del piso
    data_piso["id_url"] = piso

    # Guardar los resultados en la lista
    info_pisos.append(data_piso)

    # Generar un delay para imitar una búsqueda humana
    time.sleep(random.uniform(2, 5))

    # Aumentar el contador en 1
    contador += 1

    # Dormir la ejecución 15 segundos cuando se haya llegado
    # a un múltiplo de 100 peticiones
    if contador % 100 == 0:
        print(f"Se ha llegado a un múltiplo de 100 (durmiendo peticiones): {contador}")
        time.sleep(30)

Se ha llegado a un múltiplo de 100 (durmiendo peticiones): 100
Se ha llegado a un múltiplo de 100 (durmiendo peticiones): 200
Se ha llegado a un múltiplo de 100 (durmiendo peticiones): 300
Se ha llegado a un múltiplo de 100 (durmiendo peticiones): 400


In [49]:
# Convertir la lista de diccionarios a DataFrame
dataframe_info_pisos = pd.DataFrame(info_pisos)
dataframe_info_pisos

,title,referencia_anuncio,price,info_features,caracteristicas_básicas,certificado_energetico,Precio del inmueble:,Precio por m²:,ubicacion,id_url
0,Alquiler de Chalet adosado en Urbanització Deu...,ll-1,1.800,"[293 m², 5 hab., Garaje incluido, Garaje inclu...","[Chalet adosado, 293 m² construidos, 228 m² út...","[{'Consumo:': ['', 'icon-energy-c-g']}, {'Emis...",1.800 €/mes,"6,14 €/m²","[Urbanització Deulofeu, 3 -1, Puigcerdà, La Ce...",102140869
1,Alquiler de Piso en calle de la Font de la Teu...,105322130,1.980,"[127 m², 4 hab., Planta 2ª exterior con ascens...","[127 m² construidos, 4 habitaciones, 2 baños, ...","[{'Consumo:': ['32 kWh/m² año', 'icon-energy-c...",1.980 €/mes,"15,59 €/m²","[Calle de la Font de la Teula, 14, Distrito Fo...",105322130
2,"Alquiler de Piso en Can Sabata - Mas Baell, Ll...",104525369,1.600,"[60 m², 1 hab., Planta 5ª exterior con ascensor]","[60 m² construidos, 55 m² útiles, 1 habitación...","[{'Consumo:': ['', 'icon-energy-c-a']}, {'Emis...",1.600 €/mes,"26,67 €/m²","[Distrito Can Sabata - Mas Baell, Lloret de Ma...",104525369
3,Alquiler de Casa de pueblo en calle Sant Gaiet...,7046,1.400,"[210 m², 4 hab., Garaje incluido, Garaje inclu...","[Casa de pueblo, 3 plantas, 210 m² construidos...","[{'Consumo:': ['94 kWh/m² año', 'icon-energy-c...",1.400 €/mes,"6,67 €/m²","[Calle Sant Gaietà, 5, Boadella d'Empordà, Alt...",105113238
4,Alquiler de Casa o chalet independiente en cal...,104544516,2.000,"[230 m², 3 hab.]","[Casa o chalet independiente, 2 plantas, 230 m...","[{'Consumo:': ['', 'icon-energy-c-e']}, {'Emis...",2.000 €/mes,"8,70 €/m²","[Calle de Carles Riba s/n, Distrito Residencia...",104544516
...,...,...,...,...,...,...,...,...,...,...
463,Alquiler de Piso en Nuestra Senyora de Montser...,105169416,1.200,"[128 m², 4 hab., Planta 3ª exterior sin ascensor]","[128 m² construidos, 125 m² útiles, 4 habitaci...","[{'Consumo:': ['335 kWh/m² año', 'icon-energy-...",1.200 €/mes,"9,38 €/m²","[Nuestra Senyora de Montserrat, 6, La Pobla de...",105169416
464,"Alquiler de Piso en calle Boters, Barri Maríti...",105386962,650,"[60 m², 2 hab., Planta 2ª exterior con ascenso...","[60 m² construidos, 2 habitaciones, 1 baño, Te...","[{'En trámite': ['', 'icon-energy-c-']}]",650 €/mes,"10,83 €/m²","[Calle Boters, Distrito Barri Marítim-Platja d...",105386962
465,"Alquiler de Piso en calle de Frederic Mompou, ...",13-1-2,1.300,"[122 m², 4 hab., Planta 2ª exterior con ascens...","[122 m² construidos, 107 m² útiles, 4 habitaci...","[{'Consumo:': ['20 kWh/m² año', 'icon-energy-c...",1.300 €/mes,"10,66 €/m²","[Calle de Frederic Mompou, 13, Distrito Port, ...",104401336
466,Alquiler de Casa o chalet en calle de les Ille...,000747,650,"[160 m², 3 hab., Garaje incluido, Garaje inclu...","[Casa o chalet, 160 m² construidos, 3 habitaci...","[{'En trámite': ['', 'icon-energy-c-']}]",650 €/mes,"4,06 €/m²","[Calle de les Illes Balears, Distrito Jesús - ...",105528652


In [67]:
# Guardar el DataFrame
save_df_to_parquet(dataframe_info_pisos, "../data/raw/", "pisos", replace=True)